In [21]:
# Importation des librairies nécessaires pour la visualisation des données
import plotly.express
import pandas as pd
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor

In [22]:
#Récuperation du dataset au format csv
df= pd.read_csv("/weather.csv") # Mettre le bon lien pour le dataset

In [23]:
## Fonctions pour nettoyer des datas
# Renommer les colonnes
def data_preparation_0(df):
    for i in df.columns :
      l = i.split("_")
      if len(l)!=1:
        if len(l)>2:
          i1 = l[1]+l[2]
        else:
          i1 = l[1]
        i1 += "_"+l[0]
        df.rename(columns={i:i1},inplace=True)
    return df
# Reformatage des données
def data_preparation_1(df):
    df["id"] = df.index
    df_long = pd.wide_to_long(df.reset_index(), stubnames=[
       'precipitation', 'tempmean', 'tempmin',
       'tempmax','windspeed','pressure'], i=['DATE'], j='town',sep='_',suffix='.+')
    df_long.reset_index()
    df_long
    df3 = df_long[['precipitation', 'tempmean', 'tempmin',
       'tempmax','windspeed','pressure']]
    df3.reset_index(inplace=True)
    return df3
# Extraire les caractéristiques de la date
def extract_date_features(df):
    df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')
    df['year'] = df['DATE'].dt.year
    df['month'] = df['DATE'].dt.month
    df['day'] = df['DATE'].dt.day
    df = df.drop(columns='DATE')
    return df
# Ajouter les décalages pour les colonnes de température (tempmean et tempmax)
def day_in_Life(df, number):
    for i in range(1, number + 1):
        df[[f"tempmean{i}",f"tempmax{i}"]] = df.groupby(['town'])[["tempmean","tempmax"]].shift(i)
    return df


In [24]:
# Application des fonctions pour le nettoyage des data
df = data_preparation_0(df)
df = data_preparation_1(df)
df = extract_date_features(df)
df = day_in_Life(df,2)
df

,town,precipitation,tempmean,tempmin,tempmax,windspeed,pressure,year,month,day,tempmean1,tempmax1,tempmean2,tempmax2
0,BASEL,0.03,2.9,1.6,3.9,NaN,1.0286,2000,1,1,NaN,NaN,NaN,NaN
1,BASEL,0.00,3.6,2.7,4.8,NaN,1.0318,2000,1,2,2.9,3.9,NaN,NaN
2,BASEL,0.00,2.2,0.1,4.8,NaN,1.0314,2000,1,3,3.6,4.8,2.9,3.9
3,BASEL,0.35,3.9,0.5,7.5,NaN,1.0262,2000,1,4,2.2,4.8,3.6,4.8
4,BASEL,0.07,6.0,3.8,8.6,NaN,1.0246,2000,1,5,3.9,7.5,2.2,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62113,ROMA,NaN,6.9,2.0,11.8,NaN,1.0111,2009,12,28,6.7,10.4,9.3,11.6
62114,ROMA,NaN,12.1,11.0,13.2,NaN,1.0089,2009,12,29,6.9,11.8,6.7,10.4
62115,ROMA,NaN,14.7,12.2,17.2,NaN,1.0071,2009,12,30,12.1,13.2,6.9,11.8
62116,ROMA,NaN,14.8,12.0,17.6,NaN,1.0009,2009,12,31,14.7,17.2,12.1,13.2


In [105]:
# Pipeline
num_selector = make_column_selector(dtype_include=np.number)
num_tree_processor = SimpleImputer(strategy="mean", add_indicator=True)
tree_preprocessor = make_column_transformer((num_tree_processor, num_selector))
reg = make_pipeline(tree_preprocessor, RandomForestRegressor(n_estimators=100, random_state=42))

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f09cc46d570>)])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])
Train score 98.2311291769348
Test score 87.34118595665991


In [ ]:
x = df[['precipitation','windspeed','pressure','year', 'month', 'day']]
y = df['tempmean']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
reg = reg.fit(x_train, y_train)

print("Train score", reg.score(x_train, y_train) *100)
print("Test score", reg.score(x_test, y_test) * 100)

In [99]:
import pickle

# Supposons que vous avez un modèle nommé 'modele'
modele = {"exemple": "Ceci est un modèle d'exemple"}

# Sauvegarder le modèle dans un fichier
with open('modele.pkl', 'wb') as fichier:
    pickle.dump(reg, fichier)

print("Modèle sauvegardé avec succès.")


Modèle sauvegardé avec succès.


In [33]:
x = df[['precipitation','tempmin',
       'tempmax','windspeed','pressure', 'year', 'month', 'day']]
y = df['tempmean']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [34]:
#Modele
reg = LinearRegression()
reg.fit(x_train, y_train)

LinearRegression()

In [35]:
# Prédiction
y_pred = reg.predict(x_test)

In [37]:
# Évaluation
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Visualisation

MSE: 0.9442041491669702, MAE: 0.6490135818717973, R^2: 0.9869074909935833
